In [4]:
import os
import numpy as np
import pandas as pd
from datetime import datetime as dt
import openpyxl

from masakali_data.utils import open_csv, save_csv, current_directory, update_parquet, print_memory, open_parquet
from masakali_data.exchange_rates import convert_columns_to_usd, convert_columns_to_idr
from masakali_data.smoobu import get_reservations

current_directory = current_directory()

In [3]:
# Import the CSV
def apply_villa_name(room_type):
    switcher = {
        "Villa With Private Pool": "surya",
        "Private Pool Villa": "chandra",
        "Pool Villa": "jala",
        "Home Haven Pool Villa": "akasha",
        "2-Bedroom Superior Villa": "lakshmi",
    }

    return switcher.get(room_type, "Unknown")


def main():
    agoda = pd.DataFrame()

    for year in range(2022, 2024):
        df = open_csv(current_directory, f"{year}\\all_agoda_{year}.csv")
        agoda = pd.concat([agoda, df])

    agoda["villa_name"] = agoda["RoomType"].apply(apply_villa_name)

    relevant_columns = [
        "BookingIDExternal_reference_ID",
        "StayDateFrom",
        "StayDateTo",
        "Customer_Name",
        "ReferenceSellInclusive",
        "Commission",
        "Total_inclusive_rate",
        "villa_name",
        "BookedDate",
    ]

    # Grab relevant columns
    agoda = agoda[agoda["Status"].isin(["Confirmed", "Amended"])][relevant_columns]

    # Rename columns
    agoda = agoda.rename(
        columns={
            "Platform": "Agoda",
            "BookingIDExternal_reference_ID": "booking_id",
            "StayDateFrom": "check_in",
            "StayDateTo": "check_out",
            "Customer_Name": "guest_name",
            "ReferenceSellInclusive": "gross_income",
            "Commission": "commission",
            "Total_inclusive_rate": "net_income",
            "BookedDate": "date",
        }
    )

    agoda["platform"] = "agoda"

    # Turn check_in and check_out to datetime
    agoda["check_in"] = pd.to_datetime(agoda.check_in, format="%d-%b-%Y")
    agoda["check_out"] = pd.to_datetime(agoda.check_out, format="%d-%b-%Y")

    agoda["cleaning_fee"] = 0
    agoda["taxable_amount"] = agoda.net_income - agoda.cleaning_fee
    agoda["nights"] = (agoda.check_out - agoda.check_in).dt.days

    columns_to_convert = [
        "gross_income",
        "commission",
        "net_income",
        "cleaning_fee",
        "taxable_amount",
    ]

    agoda_usd = convert_columns_to_usd(agoda, columns_to_convert)

    agoda = agoda.drop_duplicates(subset=["booking_id"], keep="first")
    update_parquet(current_directory, agoda, f"{year}/agoda_revenue_idr.parquet")
    update_parquet(current_directory, agoda_usd, f"{year}/agoda_revenue_usd.parquet")

    # save_csv(current_directory, agoda, f"{year}/agoda_revenue_idr_{year}.csv")
    # create_monthly_sheets(
    #     current_directory, agoda, f"{year}/agoda_revenue_idr_{year}.xlsx"
    # )

    # save_csv(current_directory, agoda_usd, f"{year}/agoda_revenue_usd_{year}.csv")
    # create_monthly_sheets(
    #     current_directory, agoda_usd, f"{year}/agoda_revenue_usd_{year}.xlsx"
    # )

df = main()

34 KB
booking_id                 int64
check_in          datetime64[ns]
check_out         datetime64[ns]
guest_name                object
gross_income             float64
commission               float64
net_income               float64
villa_name                object
date                      object
platform                  object
cleaning_fee               int64
taxable_amount           float64
nights                     int64
dtype: object


In [5]:
df = open_parquet(current_directory, "data/agoda_revenue_idr.parquet")

In [6]:
df.head()

,booking_id,check_in,check_out,guest_name,gross_income,commission,net_income,villa_name,date,platform,cleaning_fee,taxable_amount,nights
0,328061479,2022-10-09,2022-10-10,Oana Boruga,1000000.0,180000.0,820000.0,jala,2022-08-19,agoda,0,820000.0,1
1,335112107,2022-11-21,2022-11-22,arnaud trioux,1700000.0,306000.0,1394000.0,surya,2022-10-12,agoda,0,1394000.0,1
2,722671853,2022-07-11,2022-07-12,Vicha Claudhya,1000000.0,180000.0,820000.0,jala,2022-06-25,agoda,0,820000.0,1
3,748663849,2022-12-07,2022-12-08,Tio rizky basania S,1300000.0,234000.0,1066000.0,jala,2022-08-05,agoda,0,1066000.0,1
4,788851748,2022-12-28,2022-12-29,Kazutomo Nakajima,1300000.0,234000.0,1066000.0,jala,2022-11-09,agoda,0,1066000.0,1
